In [1]:
from utils.mch import MCHHandler
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
wl = [i for i in range(650, 1001, 10)]

mh = MCHHandler()

mh.load_config("configs/muscle.json")

def calculate_mua(b, s, w, f, m, oxy, deoxy, water, fat, melanin):
    mua = b * (s * oxy + (1-s) * deoxy) + w * water + f * fat + m * melanin
    return mua

mua = pd.read_csv("input/coefficients.csv")
oxy = mua['oxy'].values
deoxy = mua['deoxy'].values
water = mua['water'].values
collagen = mua['collagen'].values
fat = mua['fat'].values
melanin = mua['mel'].values
wavelength = mua['wavelength'].values

# interpolation
oxy = np.interp(wl, wavelength, oxy)
deoxy = np.interp(wl, wavelength, deoxy)
water = np.interp(wl, wavelength, water)
collagen = np.interp(wl, wavelength, collagen)
fat = np.interp(wl, wavelength, fat)
melanin = np.interp(wl, wavelength, melanin)

# turn the unit 1/cm --> 1/mm
oxy *= 0.1
deoxy *= 0.1
water *= 0.1
collagen *= 0.1
fat *= 0.1
melanin *= 0.1

In [25]:

args = {
    "skin":{
        "blood_volume_fraction": 0.5,
        "ScvO2": 0.7,
        "water_volume": 0.49,
        "fat_volume": 0,
        "melanin_volume": 0.01,
    },

    "fat":{
        "blood_volume_fraction": 0,
        "ScvO2": 0,
        "water_volume": 0,
        "fat_volume": 1,
        "melanin_volume": 0,
    },

    "muscle":{
        "blood_volume_fraction": 0.05,
        "ScvO2": 0.80,
        "water_volume": 0.75,
        "fat_volume": 0,
        "melanin_volume": 0,
    }
}

spec, ratio = mh.calculate_reflectance_white(args)
for t in ["skin", "fat", "muscle"][:0]:
    mua = calculate_mua(
        args[t]["blood_volume_fraction"],
        args[t]["ScvO2"],
        args[t]["water_volume"],
        args[t]["fat_volume"],
        args[t]["melanin_volume"],
        oxy, 
        deoxy, 
        water, 
        fat, 
        melanin
    )

plt.plot(wl, mua, label="blood: %f" % i)
plt.xlabel("wavelength [nm]")
plt.ylabel("mua [1/mm]")
plt.grid()
plt.legend()
plt.show()

for i in range(5):
    plt.plot(wl, spec[i], label="%d" % i)
plt.grid()
plt.xlabel("wavelength [nm]")
plt.ylabel("reflectance [-]")
plt.legend()
plt.show()



In [34]:
spec = []
mua = []
for i in np.arange(0, 1, 0.1):
    args = {
        "skin":{
            "blood_volume_fraction": i,
            "ScvO2": 0.7,
            "water_volume": 1 - i - 0.01,
            "fat_volume": 0,
            "melanin_volume": 0.01,
        },

        "fat":{
            "blood_volume_fraction": 0,
            "ScvO2": 0,
            "water_volume": 0,
            "fat_volume": 1,
            "melanin_volume": 0,
        },

        "muscle":{
            "blood_volume_fraction": 0.05,
            "ScvO2": 0.80,
            "water_volume": 0.75,
            "fat_volume": 0,
            "melanin_volume": 0,
        }
    }

    s, ratio = mh.calculate_reflectance_white(args)
    spec.append(s)
    for t in ["skin", "fat", "muscle"][:1]:
        m = calculate_mua(
            args[t]["blood_volume_fraction"],
            args[t]["ScvO2"],
            args[t]["water_volume"],
            args[t]["fat_volume"],
            args[t]["melanin_volume"],
            oxy, 
            deoxy, 
            water, 
            fat, 
            melanin
        )
        mua.append(m)

In [46]:
plt.figure(figsize=(10, 8))
for idx, m in enumerate(mua):
    plt.plot(wl, m, label="blood: %.1f" % (idx*0.1))
plt.xlabel("wavelength [nm]")
plt.ylabel("mua [1/mm]")
plt.grid()
plt.legend()
plt.savefig("skin_blood_mua.png")
plt.clf()

plt.figure(figsize=(10, 8))
for idx, s in enumerate(spec):
    for i in range(5):
        if i == 0:
            plt.plot(wl, s[i], color="C%d" % idx, label="blood: %.1f" % (idx*0.1))
        else:
            plt.plot(wl, s[i], color="C%d" % idx)
plt.grid()
plt.title("skin")
plt.xlabel("wavelength [nm]")
plt.ylabel("reflectance [-]")
plt.legend()
plt.savefig("skin_blood_spec.png")
plt.clf()


<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

In [47]:
spec2 = []
mua2 = []
for i in np.arange(0, 1, 0.1):
    args = {
        "skin":{
            "blood_volume_fraction": 0.5,
            "ScvO2": i,
            "water_volume": 0.49,
            "fat_volume": 0,
            "melanin_volume": 0.01,
        },

        "fat":{
            "blood_volume_fraction": 0,
            "ScvO2": 0,
            "water_volume": 0,
            "fat_volume": 1,
            "melanin_volume": 0,
        },

        "muscle":{
            "blood_volume_fraction": 0.05,
            "ScvO2": 0.80,
            "water_volume": 0.75,
        }
    }

    s, ratio = mh.calculate_reflectance_white(args)
    spec2.append(s)
    for t in ["skin", "fat", "muscle"][:1]:
        m = calculate_mua(
            args[t]["blood_volume_fraction"],
            args[t]["ScvO2"],
            args[t]["water_volume"],
            args[t]["fat_volume"],
            args[t]["melanin_volume"],
            oxy, 
            deoxy, 
            water, 
            fat, 
            melanin
        )
        mua2.append(m)

In [51]:
plt.figure(figsize=(10, 8))
for idx, m in enumerate(mua2):
    plt.plot(wl, m, label="scv: %.1f" % (idx*0.1))
plt.xlabel("wavelength [nm]")
plt.ylabel("mua [1/mm]")
plt.grid()
plt.legend()
plt.savefig("skin_scv_mua.png")
plt.clf()

plt.figure(figsize=(10, 8))
for idx, s in enumerate(spec2):
    for i in range(5):
        if i == 0:
            plt.plot(wl, s[i], color="C%d" % idx, label="scv: %.1f" % (idx*0.1))
        else:
            plt.plot(wl, s[i], color="C%d" % idx)
plt.grid()
plt.title("skin")
plt.xlabel("wavelength [nm]")
plt.ylabel("reflectance [-]")
plt.legend()
plt.savefig("skin_scv_spec.png")
plt.clf()


<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>

In [43]:
def calculate_mua_muscle(b, s, w, oxy, deoxy, water, collagen):
    mua = w * water + (1-w-b) * collagen + b * (s * oxy + (1-s) * deoxy)

    return mua 

In [44]:
spec3 = []
mua3 = []
for i in np.arange(0.5, 0.8, 0.05):
    args = {
        "skin":{
            "blood_volume_fraction": 0.5,
            "ScvO2": 0.7,
            "water_volume": 0.49,
            "fat_volume": 0,
            "melanin_volume": 0.01,
        },

        "fat":{
            "blood_volume_fraction": 0,
            "ScvO2": 0,
            "water_volume": 0,
            "fat_volume": 1,
            "melanin_volume": 0,
        },

        "muscle":{
            "blood_volume_fraction": 0.05,
            "ScvO2": 0.80,
            "water_volume": i,
            "fat_volume": 0,
            "melanin_volume": 0,
        }
    }

    s, ratio = mh.calculate_reflectance_white(args)
    spec3.append(s)
    for t in ["skin", "fat", "muscle"][-1:]:
        m = calculate_mua_muscle(
            args[t]["blood_volume_fraction"],
            args[t]["ScvO2"],
            args[t]["water_volume"],
            oxy, 
            deoxy, 
            water, 
            collagen
        )
        mua3.append(m)

In [50]:
plt.figure(figsize=(10, 8))
for idx, m in enumerate(mua3):
    plt.plot(wl, m, label="water: %.1f" % (idx*0.1))
plt.xlabel("wavelength [nm]")
plt.ylabel("mua [1/mm]")
plt.grid()
plt.legend()
plt.savefig("muscle_water_mua.png")
plt.clf()

plt.figure(figsize=(10, 8))
for idx, s in enumerate(spec3):
    for i in range(5):
        if i == 0:
            plt.plot(wl, s[i], color="C%d" % idx, label="water: %.1f" % (idx*0.1))
        else:
            plt.plot(wl, s[i], color="C%d" % idx)
plt.grid()
plt.title("skin")
plt.xlabel("wavelength [nm]")
plt.ylabel("reflectance [-]")
plt.legend()
plt.savefig("muscle_water_spec.png")
plt.clf()


<Figure size 720x576 with 0 Axes>

<Figure size 720x576 with 0 Axes>